# Data Loading and Initial Inspection

This notebook loads the raw sensor data, parses the timestamps, and performs a visual inspection of the accelerometer signals.

**Goal**: Verify data integrity and visualize the raw signals.

In [43]:
import pandas as pd
import plotly.express as px
import os
import numpy as np

# Define file path
data_path = '../data/raw/20251223151337.txt'

# Check if file exists
if not os.path.exists(data_path):
    print(f"File not found: {data_path}")
else:
    print(f"File found: {data_path}")

File found: ../data/raw/20251223151337.txt


In [44]:
# Load the data
# The file is tab-separated
df = pd.read_csv(data_path, sep='\t')

# Ensure milliseconds have three digits
df['time'] = df['time'].str.replace(r'(?<=\.)(\d{1,2})$', lambda m: m.group(1).zfill(3), regex=True)

# Parse 'time' column to datetime
df['time'] = pd.to_datetime(df['time'])

# Set 'time' as index
df.set_index('time', inplace=True)

# Display the first few rows
df.head()

,DeviceName,AccX(g),AccY(g),AccZ(g),AsX(°/s),AsY(°/s),AsZ(°/s),AngleX(°),AngleY(°),AngleZ(°),...,HZ(uT),Q0(),Q1(),Q2(),Q3(),Temperature(°C),Height(m),Pressure(kPa),Version(),Battery level(%)
time,,,,,,,,,,,,,,,,,,,,,
2025-12-23 15:13:37.349,WT901BLE67(D2:B1:E5:85:1B:B2),0.184,-0.981,-0.013,-0.549,-2.869,-0.305,-89.39,-8.10,6.95,...,16.373,0.711,-0.69696,-0.09274,-0.00641,24.7,0.0,0.0,10080.1.20,10
2025-12-23 15:13:37.465,WT901BLE67(D2:B1:E5:85:1B:B2),0.146,-0.988,-0.017,-0.488,-0.671,-1.770,-89.57,-8.16,7.05,...,16.373,0.711,-0.69696,-0.09274,-0.00641,24.7,0.0,0.0,10080.1.20,10
2025-12-23 15:13:37.524,WT901BLE67(D2:B1:E5:85:1B:B2),0.218,-0.968,-0.014,-0.916,-19.409,-5.371,-89.79,-8.55,7.70,...,16.373,0.711,-0.69696,-0.09274,-0.00641,24.7,0.0,0.0,10080.1.20,10
2025-12-23 15:13:37.643,WT901BLE67(D2:B1:E5:85:1B:B2),0.187,-0.960,-0.006,-2.075,-26.123,-12.024,-90.21,-9.23,10.10,...,19.827,0.711,-0.69696,-0.09274,-0.00641,24.7,0.0,0.0,10080.1.20,10
2025-12-23 15:13:37.734,WT901BLE67(D2:B1:E5:85:1B:B2),0.121,-1.021,-0.002,4.517,-38.696,-26.306,-90.81,-11.05,13.11,...,19.827,0.711,-0.69696,-0.09274,-0.00641,24.7,0.0,0.0,10080.1.20,10


In [45]:
# Lowpass filter
from scipy.signal import butter, filtfilt
def butter_lowpass(data, cutoff, fs, order=4):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low')
    return filtfilt(b, a, data)

def lowpass_filter_series(series, cutoff=2.0, fs=20.0, order=4):
    filtered_data = butter_lowpass(series.values, cutoff, fs, order)
    return pd.Series(filtered_data, index=series.index)

# Apply lowpass filter to accelerometer data
df['accel_x_filtered'] = lowpass_filter_series(df['AccX(g)'])
df['accel_y_filtered'] = lowpass_filter_series(df['AccY(g)'])
df['accel_z_filtered'] = lowpass_filter_series(df['AccZ(g)'])

In [46]:
df['Acc_Magnitude'] = np.sqrt(df['AccX(g)']**2 + df['AccY(g)']**2 + df['AccZ(g)']**2)
df['Acc_Magnitude_filtered'] = np.sqrt(df['accel_x_filtered']**2 + df['accel_y_filtered']**2 + df['accel_z_filtered']**2)

In [47]:
# Visualize Accelerometer Data
fig = px.line(df, y=['Acc_Magnitude', 'Acc_Magnitude_filtered', 'AccX(g)', 'AccY(g)', 'AccZ(g)', 'accel_x_filtered', 'accel_y_filtered', 'accel_z_filtered'], title='Accelerometer Data over Time')
fig.show()

In [48]:
# Apply lowpass filter to gyroscope data
df['gyro_x_filtered'] = lowpass_filter_series(df['AsX(°/s)'])
df['gyro_y_filtered'] = lowpass_filter_series(df['AsY(°/s)'])
df['gyro_z_filtered'] = lowpass_filter_series(df['AsZ(°/s)'])

In [49]:
# Visualize Gyroscope (Angular Velocity) Data
# This measures how fast the sensor is rotating around each axis.
fig_gyro = px.line(df, y=['AsX(°/s)', 'AsY(°/s)', 'AsZ(°/s)', 'gyro_x_filtered', 'gyro_y_filtered', 'gyro_z_filtered'], title='Angular Velocity (Gyroscope) over Time')
fig_gyro.show()

In [50]:
# Visualize Angle (Orientation) Data
# This shows the device's orientation in space (Roll, Pitch, Yaw).
fig_angle = px.line(df, y=['AngleX(°)', 'AngleY(°)', 'AngleZ(°)'], title='Angle (Orientation) over Time')
fig_angle.show()

In [51]:
# Visualize Magnetic Field Data
fig_magnetic = px.line(df, y=['HX(uT)', 'HY(uT)', 'HZ(uT)'], title='Magnetic Field Data over Time')
fig_magnetic.show()

In [52]:
# Visualize Quaternion Data
fig_quaternion = px.line(df, y=['Q0()', 'Q1()', 'Q2()', 'Q3()'], title='Quaternion Data over Time')
fig_quaternion.show()